In [26]:
import numpy as np
import pandas as pd

In [27]:
application_record = pd.read_csv("application_record.csv")
application_record = application_record[["ID", "FLAG_OWN_CAR", "FLAG_OWN_REALTY", "CNT_CHILDREN", "AMT_INCOME_TOTAL", "NAME_INCOME_TYPE", "NAME_EDUCATION_TYPE", "NAME_FAMILY_STATUS", "NAME_HOUSING_TYPE", "DAYS_BIRTH", "DAYS_EMPLOYED", "CNT_FAM_MEMBERS"]]

application_record["FLAG_OWN_CAR"] = application_record["FLAG_OWN_CAR"].map(lambda x: x == "Y")
application_record["FLAG_OWN_REALTY"] = application_record["FLAG_OWN_REALTY"].map(lambda x: x == "Y")

application_record["CNT_FAM_MEMBERS"] = application_record["CNT_FAM_MEMBERS"].astype(int)

application_record = pd.get_dummies(application_record, columns=["NAME_INCOME_TYPE"], prefix="INCOME_TYPE")
application_record = pd.get_dummies(application_record, columns=["NAME_EDUCATION_TYPE"], prefix="EDUCATION_TYPE")
application_record = pd.get_dummies(application_record, columns=["NAME_FAMILY_STATUS"], prefix="FAMILY_STATUS")
application_record = pd.get_dummies(application_record, columns=["NAME_HOUSING_TYPE"], prefix="HOUSING_TYPE")

bool_columns = application_record.select_dtypes(include="bool").columns
application_record[bool_columns] = application_record[bool_columns].astype(int)

application_record

,ID,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,DAYS_BIRTH,DAYS_EMPLOYED,CNT_FAM_MEMBERS,INCOME_TYPE_Commercial associate,INCOME_TYPE_Pensioner,...,FAMILY_STATUS_Married,FAMILY_STATUS_Separated,FAMILY_STATUS_Single / not married,FAMILY_STATUS_Widow,HOUSING_TYPE_Co-op apartment,HOUSING_TYPE_House / apartment,HOUSING_TYPE_Municipal apartment,HOUSING_TYPE_Office apartment,HOUSING_TYPE_Rented apartment,HOUSING_TYPE_With parents
0,5008804,1,1,0,427500.0,-12005,-4542,2,0,0,...,0,0,0,0,0,0,0,0,1,0
1,5008805,1,1,0,427500.0,-12005,-4542,2,0,0,...,0,0,0,0,0,0,0,0,1,0
2,5008806,1,1,0,112500.0,-21474,-1134,2,0,0,...,1,0,0,0,0,1,0,0,0,0
3,5008808,0,1,0,270000.0,-19110,-3051,1,1,0,...,0,0,1,0,0,1,0,0,0,0
4,5008809,0,1,0,270000.0,-19110,-3051,1,1,0,...,0,0,1,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438552,6840104,0,1,0,135000.0,-22717,365243,1,0,1,...,0,1,0,0,0,1,0,0,0,0
438553,6840222,0,0,0,103500.0,-15939,-3007,1,0,0,...,0,0,1,0,0,1,0,0,0,0
438554,6841878,0,0,0,54000.0,-8169,-372,1,1,0,...,0,0,1,0,0,0,0,0,0,1
438555,6842765,0,1,0,72000.0,-21673,365243,2,0,1,...,1,0,0,0,0,1,0,0,0,0


In [28]:
credit_record = pd.read_csv("credit_record.csv")
credit_record

,ID,MONTHS_BALANCE,STATUS
0,5001711,0,X
1,5001711,-1,0
2,5001711,-2,0
3,5001711,-3,0
4,5001712,0,C
...,...,...,...
1048570,5150487,-25,C
1048571,5150487,-26,C
1048572,5150487,-27,C
1048573,5150487,-28,C


In [29]:
def group_is_good(group):
    values = {'C': 1, 'X': 0, '0': -1, '1': -2, '2': -3, '3': -4, '4': -5, '5': -6}
    threshold = -0.75
    return int(group["STATUS"].map(lambda x: values[x]).mean() >= threshold)

is_good_map = credit_record.groupby("ID").apply(group_is_good).reset_index(name="IS_GOOD")

C:\Users\moham\AppData\Local\Temp\ipykernel_28560\3770399417.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  is_good_map = credit_record.groupby("ID").apply(group_is_good).reset_index(name="IS_GOOD")


In [30]:
application_record = pd.merge(application_record, is_good_map, on="ID")
application_record

,ID,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,DAYS_BIRTH,DAYS_EMPLOYED,CNT_FAM_MEMBERS,INCOME_TYPE_Commercial associate,INCOME_TYPE_Pensioner,...,FAMILY_STATUS_Separated,FAMILY_STATUS_Single / not married,FAMILY_STATUS_Widow,HOUSING_TYPE_Co-op apartment,HOUSING_TYPE_House / apartment,HOUSING_TYPE_Municipal apartment,HOUSING_TYPE_Office apartment,HOUSING_TYPE_Rented apartment,HOUSING_TYPE_With parents,IS_GOOD
0,5008804,1,1,0,427500.0,-12005,-4542,2,0,0,...,0,0,0,0,0,0,0,1,0,1
1,5008805,1,1,0,427500.0,-12005,-4542,2,0,0,...,0,0,0,0,0,0,0,1,0,1
2,5008806,1,1,0,112500.0,-21474,-1134,2,0,0,...,0,0,0,0,1,0,0,0,0,1
3,5008808,0,1,0,270000.0,-19110,-3051,1,1,0,...,0,1,0,0,1,0,0,0,0,1
4,5008809,0,1,0,270000.0,-19110,-3051,1,1,0,...,0,1,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36452,5149828,1,1,0,315000.0,-17348,-2420,2,0,0,...,0,0,0,0,1,0,0,0,0,0
36453,5149834,0,1,0,157500.0,-12387,-1325,2,1,0,...,0,0,0,0,1,0,0,0,0,0
36454,5149838,0,1,0,157500.0,-12387,-1325,2,0,1,...,0,0,0,0,1,0,0,0,0,0
36455,5150049,0,1,0,283500.0,-17958,-655,2,0,0,...,0,0,0,0,1,0,0,0,0,0


In [31]:
application_record[["IS_GOOD"]].mean()

IS_GOOD    0.713964
dtype: float64

In [32]:
X, Y = np.array(application_record.iloc[:, 1:-1]), np.array(application_record.iloc[:, -1:])
X = np.hstack((X, np.ones((X.shape[0], 1))))